# Preprocessing

In [8]:
!pip install autocorrect

  Created wheel for autocorrect: filename=autocorrect-1.1.0-cp37-none-any.whl size=1810772 sha256=6e2dc04866ba323d7f173bd653ee8d2c6f0b2c3d2c9fe2151648cad33c9b0612
  Stored in directory: C:\Users\think\AppData\Local\pip\Cache\wheels\78\7f\b1\527522820ae623df6a2dbe14f778d23adaea4bebe43f7ebcfe
Successfully built autocorrect


In [62]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\think\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\think\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [63]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from autocorrect import Speller
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [64]:
stop_words = set(stopwords.words('english'))
spell = Speller()
ps = PorterStemmer()

In [65]:
def word_preprocess(text):
    word_tokens = word_tokenize(text)
    filtered_sentence = [ps.stem(spell(w.lower())) for w in word_tokens if spell(w.lower()) not in stop_words]
    return filtered_sentence

In [66]:
example_sent = "This is a sample sentence, showing off the stop words filtration filtrate thiz."
print(word_preprocess(example_sent))

['sampl', 'sentenc', ',', 'show', 'stop', 'word', 'filtrat', 'filtrat', '.']


# Dimensionality reduction

In [71]:
import pandas as pd
import numpy as np

In [96]:
training_set = pd.read_csv('train.csv')
dev_set = pd.read_csv('dev.csv')
for i, review in enumerate(training_set['review'][:1000]):
    if i % 100 == 0:
        print(i)
    training_set['review'][i] = " ".join(word_preprocess(review))
for i, review in enumerate(dev_set['review'][:100]):
    dev_set['review'][i] = " ".join(word_preprocess(review))

0


F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


100
200
300
400
500
600
700
800
900


F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [93]:
training_set[:10]

,ex_id,user_id,prod_id,rating,label,date,review
0,0,923,0,3.0,1,2014-12-08,food snack select popular greek dish . appet t...
1,1,924,0,3.0,1,2013-05-16,littl place soho wonder . lamb sandwich glass ...
2,2,925,0,4.0,1,2013-07-01,"order lunch 15 snack last friday . time , noth..."
3,3,926,0,4.0,1,2011-07-28,beauti quaint littl restaur pretti street . 'r...
4,4,927,0,4.0,1,2010-11-01,snack great place casual sit lunch- especi col...
5,5,928,0,4.0,1,2009-09-02,solid 4 star greek food spot . 're fan lamb li...
6,7,930,0,4.0,1,2007-05-20,love place ! tri chicken sandwich roast chicke...
7,8,931,0,4.0,1,2005-12-27,"friend intrigu nightli entre special , `` litt..."
8,10,933,0,5.0,1,2014-01-21,pretti cool place ... good food ... good peopl
9,12,935,0,5.0,1,2011-01-31,fabul authent greek food ! ! ! littl spot bold...


In [8]:
training_set.shape

(250874, 7)

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [98]:
def TFIDF(X_train, X_test, MAX_NB_WORDS=25000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with", str(np.array(X_train).shape[1]), "features")
    return (X_train, X_test)

In [159]:
X_train = training_set['review'][:1000]
X_test = dev_set['review'][:100]
y_train = training_set['label'][:1000].tolist()
y_test = dev_set['label'][:100].tolist()

X_train,X_test = TFIDF(X_train,X_test)

from sklearn.decomposition import PCA
pca = PCA(n_components=500)
X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)

print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)

tf-idf with 4736 features
train with old features:  (1000, 4736)
train with new features: (1000, 500)
test with old features:  (100, 4736)
test with new features: (100, 500)


In [160]:
print(X_train_new[:5])

[[-0.05287596 -0.05140821 -0.11206865 ...  0.03027988 -0.02944922
  -0.00789618]
 [-0.04843974 -0.08707531  0.01597805 ... -0.02225466  0.0005921
  -0.03058929]
 [-0.01883406  0.00142086 -0.02761972 ... -0.01571646  0.03474322
  -0.0036856 ]
 [-0.00071382 -0.03885944 -0.12256711 ... -0.01536525  0.01293694
   0.03259069]
 [-0.04235784 -0.05461509 -0.08903244 ...  0.06711265  0.00870736
  -0.03266734]]


In [161]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [164]:
print(y_train )

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [185]:
text_clf = Pipeline([('clf', RandomForestClassifier(n_estimators=3, class_weight='balanced'))])
text_clf.fit(X_train_new, y_train)
predicted = text_clf.predict(X_test_new)
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93        92
           1       0.22      0.25      0.24         8

    accuracy                           0.87       100
   macro avg       0.58      0.59      0.58       100
weighted avg       0.88      0.87      0.87       100

